## Thanks to:
I inspired the idea of how to load my own h5 model form:https://github.com/karansjc1/emotion-detection/blob/master/without%20flask/Test_Code.py 

I have learned how to make cv2 work on flask from: https://towardsdatascience.com/how-to-display-video-streaming-from-a-webcam-using-flask-7a15e26fbab8
 
I have known how sharing the same network techinque works from: https://www.codegrepper.com/code-examples/python/open+flask+site+on+another+device


In [ ]:
from flask import Flask, render_template, Response , request
from werkzeug.wrappers import Request, Response
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

app = Flask(__name__)
cap=cv2.VideoCapture(0)
#loading my pretrained model
classifier = tf.keras.models.load_model(('modell.h5'),custom_objects={'KerasLayer':hub.KerasLayer})
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

label_dic= {'crying':0,
           'happy':1,
           'sleeping':2}
#out varible for excuting the code just one time and avoid repeasing it through the loop
global out
out = False

@app.route('/')
def hello():   
    return render_template('hello.html')

@app.route('/app' ,methods = ['POST', 'GET'])
def index():
    global text
    text = request.form['text']
    return render_template('index.html', text = text )

def gen_frames(cap):
    while True:
        sucess, image =cap.read()
        gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        faces=face_classifier.detectMultiScale(gray,1.3,5)

        for (x,y,w,h) in faces:
            cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray=gray[y:y+h,x:x+w]

            roi_gray= cv2.cvtColor(roi_gray,cv2.COLOR_BGR2RGB)
            roi_gray=cv2.resize(roi_gray,(224,224),interpolation=cv2.INTER_AREA)
            roi=roi_gray.astype('float')/255.0
            roi=img_to_array(roi)
            roi=np.expand_dims(roi,axis=0)
            p = classifier.predict(roi)
          
            index =np.argmax(p,axis=1)
            
            def show_the_class(no):
                no=list(no)
                for i in range(len(no)):
                    for key,value in label_dic.items():
                        if no[i] == value:
                            return key

            cv2.putText(image, show_the_class(index), (x+5,y-5), cv2.FONT_HERSHEY_SIMPLEX , 1, (255,0,255), 2)
            #condition under which the email will be sent
            global out
            if show_the_class(index) == 'crying' and out == False:
                #sending email
                import smtplib
                my_email = "motheralertt@gmail.com"
                password = "the password I have used"

                subject = 'A Crying Baby'
                body = "Your baby is horribly crying!, please go to the app to keep track of him and take an action  \n The app link: IPaddressofnetwork.com "
                email_text = """\
                From: %s
                To: %s
                Subject: %s

                %s
                """ % (my_email, ", ".join(text), subject, body)
                # we used the varible text which the user already provided. 
                connection = smtplib.SMTP("smtp.gmail.com",587)
                connection.starttls()
                connection.login(user=my_email, password=password)
                connection.sendmail(my_email,text, email_text)
                print('done')
                # excuted only once and not repeated again
                out = True
                
                
        #to make cv2 run screen on flask 
        ret, jpeg = cv2.imencode('.jpg', image)
        frame = jpeg.tobytes()
        #note the varibles names which is used below
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')


@app.route('/video_feed')
def video_feed():
    global cap
    return Response(gen_frames(cap), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    #to run flask on jupyter notebook
    from werkzeug.serving import run_simple
    # 0.0.0.0 host allow all the devices sharing the same network to access the server
    run_simple('0.0.0.0', 9000, app)
       